In [104]:
%load_ext autoreload
%autoreload 2

import collections
import cv2
import numpy as np
import os
import random
import reader 
import tensorflow as tf


class simple_learner(object):
    def __init__(self, num_classes, height=28, width=28):
        self.num_classes = num_classes
        self.height = height
        self.width = width
        
        self.tf_sess = tf.InteractiveSession()
    
    def network(self, learning_rate=0.00000005):
        # input and output
        i = tf.placeholder(tf.float32, shape=[None, self.height*self.width])
        self.i = i
        l = tf.placeholder(tf.float32, shape=[None, self.num_classes])
        self.l = l
        
        # add variables
        W = tf.Variable(tf.zeros([self.height*self.width, self.num_classes]))
        b = tf.Variable(tf.zeros([self.num_classes]))
        
        #initialize
        self.tf_sess.run(tf.initialize_all_variables())
        
        # create output and loss
        output = tf.nn.softmax(tf.matmul(i, W) + b)
        self.output = output
        
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(l * tf.log(output), reduction_indices=[1]))

        # training
        self.train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
        
        correct_prediction = tf.equal(tf.argmax(l,1), tf.argmax(output,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def test_accuracy(self, test_source):
        images, labels = test_source.all()
        return self.accuracy.eval(feed_dict={self.i: images, self.l: labels})
        
    def train(self, num_iterations, train_source, test_source):
        for i in xrange(num_iterations):
            if i % 500 == 0: 
                print i, self.test_accuracy(test_source)

            images, labels = train_source.next_batch(50)
            self.train_step.run(feed_dict={self.i: images, self.l: labels})
        
        print self.test_accuracy(test_source)
        
    def inference(self, images):
        return self.output.eval(feed_dict={self.i: images})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
train = reader.reader('/home/lee/Desktop/rooms')
test = reader.reader('/home/lee/Desktop/rooms_test')


imgHeight = 28
imgWidth = 28
numClasses = train.num_classes()

learner = simple_learner(numClasses)
learner.network()
learner.train(2000, train, test)

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fc9b0957b50>> ignored


0 0.333333
500 0.555556
1000 0.555556
1500 0.577778
0.6


In [108]:
print "----TRAINING----"
train.print_classes()
print "----TESTING----"
test.print_classes()
print ""

from sklearn.metrics import confusion_matrix

images, labels = test.all()
output = tf.argmax(learner.inference(images), 1).eval()
labels = tf.argmax(labels, 1).eval()


print "----Confusion Matrix----"
confusion_matrix(output, labels)

----TRAINING----
dining_room 42
kitchen 44
living_room 57
----TESTING----
dining_room 15
kitchen 15
living_room 15

----Confusion Matrix----


array([[ 6,  2,  7],
       [ 4, 13,  0],
       [ 5,  0,  8]])

In [ ]:
# # input and output
# x = tf.placeholder(tf.float32, shape=[None, imgHeight*imgWidth])
# y_ = tf.placeholder(tf.float32, shape=[None, numClasses])

# # create network
# W = tf.Variable(tf.zeros([imgHeight*imgWidth,numClasses]))
# b = tf.Variable(tf.zeros([numClasses]))
# sess.run(tf.initialize_all_variables())
# y = tf.nn.softmax(tf.matmul(x,W) + b)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


# # training
# train_step = tf.train.GradientDescentOptimizer(0.00000005).minimize(cross_entropy)

# correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# for i in xrange(10000):
#     if (i) % 500 == 0: 
#         images, labels = test.all()
#         print i, (accuracy.eval(feed_dict={x: images, y_: labels}))
        
#     batch = train.next_batch(50)
#     train_step.run(feed_dict={x: batch[0], y_: batch[1]})
#     #print y.eval(feed_dict={x: batch[0]})
    

# images, labels = test.all()
# print(accuracy.eval(feed_dict={x: images, y_: labels}))

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


# W_conv1 = weight_variable([5, 5, 1, 32])
# b_conv1 = bias_variable([32])

# x_image = tf.reshape(x, [-1,28,28,1])

# h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# h_pool1 = max_pool_2x2(h_conv1)

# W_conv2 = weight_variable([5, 5, 32, 64])
# b_conv2 = bias_variable([64])

# h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
# h_pool2 = max_pool_2x2(h_conv2)


# W_fc1 = weight_variable([7 * 7 * 64, 1024])
# b_fc1 = bias_variable([1024])

# h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# keep_prob = tf.placeholder(tf.float32)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# W_fc2 = weight_variable([1024, 10])
# b_fc2 = bias_variable([10])

# y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
# train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# sess.run(tf.initialize_all_variables())
# for i in range(20000):
#   batch = mnist.train.next_batch(50)
#   if i%100 == 0:
#     train_accuracy = accuracy.eval(feed_dict={
#         x:batch[0], y_: batch[1], keep_prob: 1.0})
#     print("step %d, training accuracy %g"%(i, train_accuracy))
#   train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# print("test accuracy %g"%accuracy.eval(feed_dict={
#     x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))